

| 인덱스 알고리즘 | 시간 복잡도 (Time Complexity) | 핵심 변수 및 설명 |
| :--- | :--- | :--- |
| **FLAT** (전수조사) | $O(N \cdot D)$ | • **$N$**: 데이터 수, **$D$**: 차원 수<br>• 데이터가 2배 늘면 시간도 2배 걸림 (선형 증가).<br>• $N$이 작을 땐 오버헤드가 없어 가장 빠름. |
| **IVF** (공간분할) | $O(\sqrt{N} \cdot D)$ | • **$\sqrt{N}$**: $nlist \approx \sqrt{N}$ 로 튜닝했을 때의 이론적 속도.<br>• 실제로는 **$\frac{N}{nlist} \cdot nprobe$** 만큼 연산함.<br>• 정확도를 높이려 $nprobe$를 키우면 속도가 급격히 느려짐. |
| **HNSW** (계층 그래프) | $O(\log N \cdot D)$ | • **$\log N$**: 그래프의 계층(Layer) 높이.<br>• 데이터가 100배, 1000배 늘어도 시간은 아주 조금만 늘어남.<br>• 정확도(Recall)를 높여도 속도 저하가 적음. |



In [1]:
import psutil
import gc
import os
import numpy as np
import faiss

In [2]:
def get_memory_usage():
    """현재 파이썬 프로세스가 실제로 점유 중인 메모리(MB) 반환"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)


def get_vectors(num, dim):
    print(f'벡터 생성 전 메모리 : {get_memory_usage():.2f}MB')
    vectors = np.random.random((num, dim)).astype('float32')
    print(f'<<{dim}차원 벡터 {num}개>>')
    print(f'벡터 생성 후 메모리 : {get_memory_usage():.2f}MB')
    return vectors

In [3]:
DIMS = [768, 1536, 3072]
NUMS = [10000, 50000, 100000, 250000, 500000]
dim = 0
num = 0
q_num = 100

In [6]:
%%capture results

vectors = get_vectors(NUMS[num], DIMS[dim])
query_vector = np.random.random((q_num, DIMS[dim])).astype('float32') # 100개 배치 처리
print()

# index flat
print('==== INDEX FLAT L2 ====')
print(f'인덱스 생성 전 메모리 : {get_memory_usage():.2f}MB')
index_flat = faiss.IndexFlatL2(DIMS[dim])
print('Flat ADD 시간')
%time index_flat.add(vectors)
print()

del vectors
gc.collect()

print(f'인덱스 생성 후 메모리 : {get_memory_usage():.2f}MB')

# 웜업 
index_flat.search(query_vector, k=5)
print('Flat 검색 시간')
%time D, I = index_flat.search(query_vector, k=5)
print()

del index_flat, query_vector
gc.collect()



In [12]:
with open('results_flat_768.txt', 'a', encoding='utf-8') as f:
    f.write(results.stdout)
    

In [9]:
%%capture results

vectors = get_vectors(NUMS[num], DIMS[dim])
query_vector = np.random.random((q_num, DIMS[dim])).astype('float32') # 100개 배치 처리
print()

# ivf flat
print('==== IVF FLAT ====')
print(f'인덱스 생성 전 메모리 : {get_memory_usage():.2f}MB')

nlist = 4 * int(np.sqrt(NUMS[num])) # 4 x sqrt(n)
print(f'nlist: {nlist}')
nprobe = 16 # 90~95% 이상의 재현율. 해당 문서 참고 (https://www.emergentmind.com/topics/milvus-hnsw-ivf)
# nprobe = int(nlist * 0.031) # Mlivus IVF 문서 참고(https://milvus.io/docs/ko/v2.4.x/performance_faq.md)
print(f'nprobe: {nprobe}')

quantizer = faiss.IndexFlatL2(DIMS[dim]) 
index_ivf = faiss.IndexIVFFlat(quantizer, DIMS[dim], nlist)
index_ivf.nprobe = nprobe

print('IVF 학습 시간')
%time index_ivf.train(vectors)
print()

print('IVF ADD 시간')
%time index_ivf.add(vectors)
print()

del vectors
gc.collect()

print(f'인덱스 생성 후 메모리 : {get_memory_usage():.2f}MB')
print()

# 웜업
index_ivf.search(query_vector, k=5)

print('IVF 검색 시간')
%time index_ivf.search(query_vector, k=5)
print()

del index_ivf, quantizer, query_vector
gc.collect()


In [11]:
with open('results_ivf_low_np.txt', 'a', encoding='utf-8') as f:
    f.write(results.stdout)


In [10]:
%%capture results

vectors = get_vectors(NUMS[num], DIMS[dim])
query_vector = np.random.random((q_num, DIMS[dim])).astype('float32') # 100개 배치 처리
print()

# hnsw 
print('==== HNSW ====')
print(f'인덱스 생성 전 메모리 : {get_memory_usage():.2f}MB')
M = 32
ef_construction = 64
ef_search = 100

index_hnsw = faiss.IndexHNSWFlat(DIMS[dim], M)
index_hnsw.hnsw.efConstruction = ef_construction

print('HNSW ADD 시간')
%time index_hnsw.add(vectors)
print()

del vectors
gc.collect()

print(f'인덱스 생성 후 메모리 : {get_memory_usage():.2f}MB')
print()

index_hnsw.hnsw.efSearch = ef_search

# 웜업
index_hnsw.search(query_vector, k=5)

print('HNSW 검색 시간')
%time index_hnsw.search(query_vector, k=5)
print()

del index_hnsw, query_vector


In [5]:
with open('results_hnsw_768.txt', 'a', encoding='utf-8') as f:
    f.write(results.stdout)
